In [ ]:
import polars   as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
from  rtsvg                   import PolarsSpringLayout
from  linknode_graph_patterns import LinkNodeGraphPatterns
import random
import copy
import time
def graphToDataFrame(_g_):
    _lu_ = {'fm':[], 'to':[]}
    for _node_ in _g_.nodes(): 
        for _nbor_ in _g_.neighbors(_node_):
            _lu_['fm'].append(_node_), _lu_['to'].append(_nbor_)
    return pl.DataFrame(_lu_), [('fm','to')]

_patterns_ = LinkNodeGraphPatterns()
_tiles_    = []
for _type_ in _patterns_.types:
    _g_             = _patterns_.createPattern(_type_)
    _df_, _relates_ = graphToDataFrame(_g_)
    t0 = time.time()
    _pos1_ = nx.spring_layout(_g_)
    t1 = time.time()
    _pos2_ = rt.springLayout(_g_, use_weights=True)
    t2 = time.time()
    _pos3_ = PolarsSpringLayout(_g_).results()
    t3 = time.time()
    _pos4_ = PolarsSpringLayout(_g_, stress_threshold=None).results()
    t4 = time.time()
    _tiles_.append(rt.addTitleToSVG(rt.link(_df_, _relates_, _pos1_), f'{_type_} | NetworkX | {t1-t0:0.2f}s'))
    _tiles_.append(rt.addTitleToSVG(rt.link(_df_, _relates_, _pos2_), f'{_type_} | RT Ref | {t2-t1:0.2f}s'))
    _tiles_.append(rt.addTitleToSVG(rt.link(_df_, _relates_, _pos3_), f'{_type_} | Polars | {t3-t2:0.2f}s'))
    _tiles_.append(rt.addTitleToSVG(rt.link(_df_, _relates_, _pos4_), f'{_type_} | Polars (No Stress Check) | {t4-t3:0.2f}s'))
rt.table(_tiles_, per_row=4, spacer=10)

In [ ]:
_psl_ = PolarsSpringLayout(_patterns_.createPattern('mesh'), iterations=512)
rt.tile([_psl_.svgAnimation(draw_links=True, r=0.3)])

In [ ]:
_df_stress_ = pl.DataFrame({'stress':_psl_.stressSums()}).with_columns(pl.int_range(1, pl.len() + 1).alias("id"))
rt.xy(_df_stress_, x_field='id', y_field='stress', dot_size='small', w=1024)

In [ ]:
_df_stress_diff_ = _df_stress_.with_columns((pl.col('stress') - pl.col('stress').shift(-1)).alias('stress_diff')).drop_nulls()
rt.xy(_df_stress_diff_, x_field='id', y_field='stress_diff', dot_size='small', w=1024)